In [1]:
# from structures import Block, Disk
# from utils import *

In [2]:
# data = parse_data()
# sort by averageRating
# data.sort(key=lambda record: record[1]) 

In [3]:
# for record in data:
#     tconst, average_rating, num_votes = record
#     tconst_bytes = convert_string_to_bytes(tconst, 10)
#     average_rating_bytes = convert_float_to_bytes(average_rating)
#     num_votes_bytes = convert_uint_to_bytes(num_votes)
#     assert(convert_bytes_to_string(tconst_bytes) == tconst)
#     assert(convert_bytes_to_float(average_rating_bytes) == average_rating)
#     assert(convert_bytes_to_uint(num_votes_bytes) == num_votes)

In [4]:
# test_block = Block()
# set_data_block_header(test_block, 7)
# get_data_block_header(test_block)
# get_block_type(test_block)

In [5]:
# test_block = Block()
# set_index_block_header(test_block, 23, 22, "root")
# get_index_block_header(test_block)
# get_block_type(test_block)

In [6]:
# test_block = Block()
# set_index_block_header(test_block, 23, 22, "leaf")
# get_index_block_header(test_block)
# get_block_type(test_block)

In [7]:
# data[:2]

In [8]:
# test_block = Block()
# set_data_block_header(test_block, 3)
# insert_record_bytes(test_block, convert_record_to_bytes(data[0]))
# insert_record_bytes(test_block, convert_record_to_bytes(data[1]))

In [9]:
# print(convert_bytes_to_record(test_block.bytes[9:27]))
# print(convert_bytes_to_record(test_block.bytes[27:27+18]))

In [10]:
# while insert_record_bytes(test_block, convert_record_to_bytes(data[0])):
#     pass

In [11]:
# for i in range(9, 100, 18):
#     if i + 18 <= 100:
#         print(convert_bytes_to_record(read_record_bytes(test_block, i)))

In [12]:
# block = Block()

In [13]:
# set_index_block_header(block, 0, 0, "root")

In [14]:
# get_index_block_header(block)

In [15]:
# pointers, keys = deserialize_index_block(block)

In [16]:
# pointers, keys

In [17]:
# pointers.append((3, 100))
# keys.append(5)

In [18]:
# serialized = serialize_index_block(pointers, keys)
# print(len(serialized))

In [19]:
# set_key_pointers_bytes(block, serialized)

In [20]:
# serialize_index_block(pointers, keys)

In [21]:
# print(block)

In [22]:
# deserialize_index_block(block)

In [23]:
# to deal with duplicate key -> record header need to have pointer (block_and_offset) of next record with same average_rating

In [97]:
class BPTree:
    
    def __init__(self):
        self.root = BPTreeNode(True)
    
    def insert(self, key, value):
        self.root = self.root.insert(key, value)
        
    def search(self, key):
        return self.root.search(key)
    
    def search_range(self, lower, upper):
        return self.root.search_range(lower, upper)
    
    def delete(self, key):
        if not self.search(key):
            return
        self.root.delete(key, None, None, None, float("inf"), float("-inf"))
        if self.root.keys == []:
            self.root = self.root.pointers[0]
            
    def validate(self):
        self.root.validate()
        
    def show(self):
        # bfs
        cur = [self.root]
        while True:
            nxt = []
            to_print = []
            for node in cur:
                if node == None:
                    print("Empty tree")
                    return
                for key in node.keys:
                    to_print.append(key)
                to_print.append("|")
                for pointer in node.pointers:
                    nxt.append(pointer)
            print(" ".join(str(x) for x in to_print))
            print()
            if any(type(x) == str for x in nxt):
                return
            cur = nxt
        

In [98]:
class BPTreeNode:
    
    def __init__(self, leaf=False):
        self.keys = []
        self.pointers = [None] # either val or node
        self.leaf = leaf
        self.max_keys = 3
        self.min_leaf_keys = (self.max_keys + 1) // 2
        self.min_non_leaf_keys = self.max_keys // 2
        
    def insert(self, key, value):
        if self.leaf:
            inserted = False
            for i in range(len(self.keys)):
                if key < self.keys[i]:
                    self.keys.insert(i, key)
                    self.pointers.insert(i, value)
                    inserted = True
                    break
            if not inserted:
                pos = len(self.keys)
                self.keys.insert(pos, key)
                self.pointers.insert(pos, value)
            if len(self.keys) > self.max_keys:
                num_left = (len(self.keys) + 1) // 2
                right_node = BPTreeNode(leaf=True)
                right_node.keys = self.keys[num_left:]
                right_node.pointers = self.pointers[num_left:]
                self.keys = self.keys[:num_left]
                self.pointers = self.pointers[:num_left]
                self.pointers.append(right_node)
                above_node = BPTreeNode()
                above_node.keys = [right_node.keys[0]]
                above_node.pointers = [self, right_node]
                return above_node
            return self
        
        else:
            # pos to insert
            pos = None
            for i in range(len(self.keys)):
                if key < self.keys[i]:
                    pos = i
                    break
            if pos == None:
                pos = len(self.keys)
            to_merge = self.pointers[pos].insert(key, value)
            if to_merge is self.pointers[pos]:
                return self
            self.pointers[pos] = to_merge.pointers[1]
            self.pointers.insert(pos, to_merge.pointers[0])
            self.keys.insert(pos, to_merge.keys[0])
            if len(self.keys) > self.max_keys:
                num_left = len(self.keys) // 2
                
                right_node = BPTreeNode()
                right_node.keys = self.keys[num_left+1:]
                right_node.pointers = self.pointers[num_left+1:]
                
                above_node = BPTreeNode()
                above_node.keys = [self.keys[num_left]]
                above_node.pointers = [self, right_node]
                
                self.keys = self.keys[:num_left]
                self.pointers = self.pointers[:num_left+1]
                
                return above_node
            return self
        
    def search(self, key):
        if self.leaf:
            for i in range(len(self.keys)):
                if key == self.keys[i]:
                    return self.pointers[i]
            return None
        else:
            for i in range(len(self.keys)):
                if key < self.keys[i]:
                    return self.pointers[i].search(key)
            return self.pointers[-1].search(key)
    
    def search_first_gte(self, key):
        if self.leaf:
            for i in range(len(self.keys)):
                if self.keys[i] >= key:
                    return self, i
            if self.pointers[-1] == None:
                return None
            return self.pointers[-1], 0
        else:
            for i in range(len(self.keys)):
                if key < self.keys[i]:
                    return self.pointers[i].search_first_gte(key)
            return self.pointers[-1].search_first_gte(key)
        
    
    def search_range(self, lower, upper): # inclusive
        if lower == None and upper == None:
            raise Exception("At least one must be not None")
        if lower == None:
            lower = float("-inf")
        if upper == None:
            upper = float("inf")
        if lower > upper:
            return []
        first_gte = self.search_first_gte(lower)
        
        res = []
        if first_gte == None:
            return res
        node, pos = first_gte
        while node:
            for i in range(pos, len(node.keys)):
                if node.keys[i] > upper:
                    return res
                res.append(node.pointers[i])
            pos = 0
            node = node.pointers[-1]
        return res
    
    def delete(self, key, left_neighbour, right_neighbour, parent, upper_bound, lower_bound):
        """
        different return values have different meaning
        if return [0] => the function was called on a leaf node, and there is no merging done
        if return [1, 0] => the function was called on a leaf node and that leaf node merged with its right sibling
        if return [1, 1] => the function was called on a leaf node and that leaf node merged with its left sibling
        if return [2, x] => the function was called on a non-leaf node and there was no merging done,
            x is the minimum value in the tree rooted at that non-leaf node
        if return [3, 0] => the function was called on a non-leaf node and that non-leaf node merged with its right sibling
        if return [3, 1] => the function was called on a non-leaf node and that non-leaf node merged with its left sibling
        """
        if self.leaf:
            for i in range(len(self.keys)):
                if self.keys[i] == key:
                    self.keys.pop(i)
                    self.pointers.pop(i)
                    break
            # at this point, the pointer and key in leaf node is removed
            # since min leaf key is satisfied, return False is signify no merging is done
            if len(self.keys) >= self.min_leaf_keys:
                return [0]
            # we know min leaf key is not satisfied
            if parent == None:
                # however, since its the root, its ok
                return [0]
            # check if can take from left neighbour
            if left_neighbour and len(left_neighbour.keys) > self.min_leaf_keys:
                self.keys = [left_neighbour.keys.pop()] + self.keys
                self.pointers = [left_neighbour.pointers.pop(-2)] + self.pointers
                return [0]
            # check if can take from right neighbour
            if right_neighbour and len(right_neighbour.keys) > self.min_leaf_keys:
                self.keys += [right_neighbour.keys.pop(0)]
                self.pointers = self.pointers[:-1] + [right_neighbour.pointers.pop(0)] + [self.pointers[-1]]
                return [0]
            # if neither works, merge
            # try merge right
            if right_neighbour:
                assert len(self.keys) + len(right_neighbour.keys) <= self.max_keys
                self.pointers.pop()
                self.keys += right_neighbour.keys
                self.pointers += right_neighbour.pointers
                return [1, 0]
            # try merge left
            elif left_neighbour:
                assert len(self.keys) + len(left_neighbour.keys) <= self.max_keys
                self.keys = left_neighbour.keys + self.keys
                self.pointers = left_neighbour.pointers[:-1] + self.pointers
                return [1, 1]
            else:
                raise Exception("FUCKED")
        else:
            pos = None
            for i in range(len(self.keys)):
                if key < self.keys[i]:
                    pos = i
                    break
            if pos == None:
                pos = len(self.keys)
            # pointers[pos] is the node we recursively call on
            # get the left and right sibling
            left_sibling = self.pointers[pos-1] if pos-1 >= 0 else None
            right_sibling = self.pointers[pos+1] if pos+1 < len(self.pointers) else None
            if 0 <= pos < len(self.keys):
                call_upper_bound = self.keys[pos]
            else:
                call_upper_bound = upper_bound
            if 0 <= pos - 1 < len(self.keys):
                call_lower_bound = self.keys[pos-1]
            else:
                call_lower_bound = lower_bound
            # res is the result from the recursive call
            res = self.pointers[pos].delete(key, left_sibling, right_sibling, self, call_upper_bound, call_lower_bound)
            if res[0] == 0:
                # res == 0 means that there was no merging from leaves
                # simply need to update the value of keys[pos-1] and keys[pos]
                if pos < len(self.keys):
                    self.keys[pos] = self.pointers[pos+1].keys[0]
                if pos-1 >= 0:
                    self.keys[pos-1] = self.pointers[pos].keys[0]
                return 2, self.pointers[0].keys[0]
            elif res[0] == 2:
                if pos-1 >= 0:
                    self.keys[pos-1] = res[1]
                return 2, res[1]
            else:
                if res[1] == 0:
                    # below merged right
                    # delete self.pointers[pos+1] and self.keys[pos]
                    assert pos + 1 < len(self.pointers)
                    self.pointers.pop(pos+1)
                    assert pos < len(self.keys)
                    self.keys.pop(pos)
                elif res[1] == 1:
                    # below merged left
                    # delete self.pointers[pos-1] and self.keys[pos-1]
                    self.pointers.pop(pos-1)
                    self.keys.pop(pos-1)
                # check if non leaf min keys is satisfied
                if len(self.keys) >= self.min_non_leaf_keys or parent == None:
                    return 2, self.pointers[0].keys[0]
                # borrow from left
                if left_neighbour and len(left_neighbour.keys) > self.min_non_leaf_keys:
                    self.keys = [left_neighbour.keys.pop()] + self.keys
                    self.pointers = [left_neighbour.pointers.pop()] + self.pointers
                    self.keys[0] = self.pointers[1].keys[0]
                    return 2, self.pointers[0].keys[0]
                # borrow from right
                if right_neighbour and len(right_neighbour.keys) > self.min_non_leaf_keys:
                    self.keys += [right_neighbour.keys.pop(0)]
                    self.pointers += [right_neighbour.pointers.pop(0)]
                    self.keys[-1] = self.pointers[-1].keys[0]
                    return 2, self.pointers[0].keys[0]
                # if neither works, merge
                if right_neighbour:
                    assert upper_bound != float("inf")
                    self.keys += [upper_bound] + right_neighbour.keys
                    self.pointers += right_neighbour.pointers
                    return [3, 0]
                elif left_neighbour:
                    assert lower_bound != float("-inf")
                    self.keys = left_neighbour.keys + [lower_bound] + self.keys
                    self.pointers = left_neighbour.pointers + self.pointers
                    return [3, 1]
                else:
                    raise Exception("FUCKED")
    
    def validate(self):
        # assume that the tree has >1 node
        # check number of keys and pointers are valid
        if self.leaf:
            assert self.min_leaf_keys <= len(self.keys) <= self.max_keys
        else:
            assert self.min_non_leaf_keys <= len(self.keys) <= self.max_keys
        assert len(self.pointers) - len(self.keys) == 1
        # check that values in a node are sorted
        for i in range(len(self.keys) - 1):
            assert self.keys[i] <= self.keys[i+1]
        if type(self.pointers[0]):
            # already reach leaf nodes
            return True
        # check that values in a level are sorted
        for i in range(len(self.pointers)-1):
            assert self.pointers[i].keys[0] <= self.pointers[i+1].keys[0]
        # recursively check
        for i in range(len(self.pointers)):
            assert validate_tree(self.pointers[i])

In [99]:
tree = BPTree()

In [100]:
mock_data = {x: f"{x}x" for x in [5, 20, 35, 1, 2, 5, 6, 9, 10, 18, 19, 11, 12]}

In [101]:
for k, v in mock_data.items():
    tree.insert(k, v)
for k, v in mock_data.items():
    assert tree.search(k) == v

In [102]:
tree.show()

18 |

5 9 11 | 20 |

1 2 | 5 6 | 9 10 | 11 12 | 18 19 | 20 35 |



In [103]:
tree.delete(35)
tree.show()
tree.validate()

11 |

5 9 | 18 |

1 2 | 5 6 | 9 10 | 11 12 | 18 19 20 |



In [104]:
tree = BPTree()

In [105]:
mock_data = {x: f"{x}x" for x in [5,20,35,1,2,5,6,9,10,18,19,11,12,22,24,25,31,32,34,35,37,38]}

In [106]:
for k, v in mock_data.items():
    tree.insert(k, v)
for k, v in mock_data.items():
    assert tree.search(k) == v

In [107]:
tree.show()

18 31 |

5 9 11 | 20 24 | 34 37 |

1 2 | 5 6 | 9 10 | 11 12 | 18 19 | 20 22 | 24 25 | 31 32 | 34 35 | 37 38 |



In [108]:
tree.delete(25)

In [109]:
tree.show()

18 31 |

5 9 11 | 20 | 34 37 |

1 2 | 5 6 | 9 10 | 11 12 | 18 19 | 20 22 24 | 31 32 | 34 35 | 37 38 |



In [110]:
tree.delete(24)
tree.show()

18 31 |

5 9 11 | 20 | 34 37 |

1 2 | 5 6 | 9 10 | 11 12 | 18 19 | 20 22 | 31 32 | 34 35 | 37 38 |



In [111]:
tree.delete(22)
tree.show()

11 31 |

5 9 | 18 | 34 37 |

1 2 | 5 6 | 9 10 | 11 12 | 18 19 20 | 31 32 | 34 35 | 37 38 |



In [112]:
tree.validate()